In [1]:
import numpy

In [2]:
import tensorflow

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
import main, pickle

In [6]:
random_board_pairs = []
with open("2021-09-18-mcts-v3-boards13-random.pickle", "rb") as f:
    try:
        while True:
            random_board_pairs.append(pickle.load(f))
    except EOFError:
        pass

In [7]:
len(random_board_pairs)

20156

In [4]:
game_board_pairs = []
with open("2021-10-12-mcts-v3-boards14-games.pickle", "rb") as f:
    try:
        while True:
            game_board_pairs.append(pickle.load(f))
    except EOFError:
        pass

In [5]:
len(game_board_pairs)

21652

In [8]:
random_inputs = []
random_outputs = []
for p in random_board_pairs:
    random_inputs.append(p[0].get_flipped_3d_array())
    random_outputs.append(float(p[1]))

In [9]:
game_inputs = []
game_outputs = []
for p in game_board_pairs:
    game_inputs.append(p[0].get_flipped_3d_array())
    game_outputs.append(float(p[1]))

In [10]:
import numpy as np

In [11]:
game_inputs = np.array(game_inputs)
random_inputs = np.array(random_inputs)
game_outputs = np.array(game_outputs)
random_outputs = np.array(random_outputs)

In [12]:
random_mark = int(len(random_outputs)*0.8)

In [13]:
game_mark = int(len(game_outputs)*0.8)

In [14]:
print(random_mark, game_mark)

16124 17321


In [15]:
train_inputs = np.concatenate((random_inputs[0:random_mark] , game_inputs[0:game_mark] 
                               , random_inputs[random_mark:] ,  game_inputs[game_mark:])) 

In [16]:
train_outputs = np.concatenate(( random_outputs[0:random_mark],
                                 game_outputs[0:game_mark],
                                random_outputs[random_mark:] ,
                                game_outputs[game_mark:] ))

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [18]:
model2 = keras.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [19]:
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(1, activation='relu'))

In [20]:
model2.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [21]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 10560)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               2703616   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,722,977
Trainable params: 2,722,977
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1046/1046 [==============================] - 25s 23ms/step - loss: 0.1010 - val_loss: 0.0582
Epoch 2/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0552 - val_loss: 0.0536
Epoch 3/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0505 - val_loss: 0.0580
Epoch 4/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0477 - val_loss: 0.0495
Epoch 5/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0452 - val_loss: 0.0488
Epoch 6/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0429 - val_loss: 0.0498
Epoch 7/10
1046/1046 [==============================] - 25s 24ms/step - loss: 0.0406 - val_loss: 0.0517
Epoch 8/10
1046/1046 [==============================] - 29s 28ms/step - loss: 0.0387 - val_loss: 0.0504
Epoch 9/10
1046/1046 [==============================] - 31s 29ms/step - loss: 0.0370 - val_loss: 0.0484
Epoch 10/10
1046/1046 [==============================] - 30s 28m

When training v3 (which did better than v2), CV loss was about 0.029-30. 

When training v4 (which wasn't an improvement and in fact might have done worse than v3), CV loss was about 0.028-29

When training v5 (which didn't do better than neural net v3), CV loss was about 0.048
                                                                                 
Now v7's CV loss has plateaued at 0.049. Hmm. Maybe that's just a reflection of the fact that v5 and v7's data came from MCTS? I hope the data isn't messy

MCTS5 was supposed to generate less messy data, but I nixed that because it lost to MCTS3

(Note: I trained options for a v6, but never saved one bc the CV lossses didn't look better than for v5, and it was the same data anyway)

In [23]:
model2.save('saved_model/2021-10-26-v7-model')

INFO:tensorflow:Assets written to: saved_model/2021-10-26-v7-model/assets


Maybe I need pooling?? Or an extra Conv2d layer? 

Model3 has an extra conv2d layer

In [16]:
model3 = keras.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [17]:
model3.add(layers.Flatten())
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(1, activation='relu'))

In [19]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 9, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 7488)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1917184   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 1,973,473
Trainable params: 1,973,473
Non-trainable params: 0
______________________________________________

I have fewer parameters? Because the last conv layer made the array narrower than it was tall, so I didn't need as much dense?

In [21]:
model3.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [21]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.1007 - val_loss: 0.0646
Epoch 2/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0521 - val_loss: 0.0523
Epoch 3/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0477 - val_loss: 0.0491
Epoch 4/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0441 - val_loss: 0.0503
Epoch 5/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0418 - val_loss: 0.0490
Epoch 6/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0392 - val_loss: 0.0468
Epoch 7/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0372 - val_loss: 0.0497
Epoch 8/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0350 - val_loss: 0.0485
Epoch 9/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0327 - val_loss: 0.0459
Epoch 10/10
1136/1136 [==============================] - 23s 20m

When training v4 (and the others, I think), the CV loss would be about 0.029. Now, when training v5, it's about 0.047. That's 62% higher. 

Maybe that's a good sign—the data is more complicated and matches the real world better, so the neural net has to struggle to learn it.

Maybe that's a bad sign—the data has a lot of random noise, so the net has to struggle to learn it

Aiming for below 0.047 CV loss

In [23]:
model3.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 35s 30ms/step - loss: 0.1009 - val_loss: 0.0562
Epoch 2/10
1136/1136 [==============================] - 34s 30ms/step - loss: 0.0521 - val_loss: 0.0528
Epoch 3/10
1136/1136 [==============================] - 35s 31ms/step - loss: 0.0462 - val_loss: 0.0440
Epoch 4/10
1136/1136 [==============================] - 52s 46ms/step - loss: 0.0425 - val_loss: 0.0439
Epoch 5/10
1136/1136 [==============================] - 40s 35ms/step - loss: 0.0404 - val_loss: 0.0431
Epoch 6/10
1136/1136 [==============================] - 42s 37ms/step - loss: 0.0386 - val_loss: 0.0435
Epoch 7/10
1136/1136 [==============================] - 43s 38ms/step - loss: 0.0367 - val_loss: 0.0440
Epoch 8/10
1136/1136 [==============================] - 45s 40ms/step - loss: 0.0354 - val_loss: 0.0401
Epoch 9/10
1136/1136 [==============================] - 45s 39ms/step - loss: 0.0342 - val_loss: 0.0407
Epoch 10/10
1136/1136 [==============================] - 46s 40m

Hmm, so CV loss is 0.041. 41e-2 is better than 47e-2, but I want to aim for 29e-2

Let's add pooling between the Conv2Ds, like this: https://www.tensorflow.org/tutorials/images/cnn

In [27]:
model4 = keras.Sequential()
model4.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model4.add(layers.MaxPooling2D((2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [28]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 17, 13, 32)        608       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 6, 32)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 6, 4, 64)          18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 2, 64)          36928     
Total params: 56,032
Trainable params: 56,032
Non-trainable params: 0
_________________________________________________________________


In [29]:
model4.add(layers.Flatten())
model4.add(layers.Dense(256, activation='relu'))
model4.add(layers.Dense(1, activation='relu'))

In [30]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 17, 13, 32)        608       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 6, 32)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 6, 4, 64)          18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 2, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

Pooling makes the model smaller

In [32]:
model4.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [33]:
model4.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.1170 - val_loss: 0.0633
Epoch 2/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0611 - val_loss: 0.0595
Epoch 3/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0556 - val_loss: 0.0546
Epoch 4/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0526 - val_loss: 0.0585
Epoch 5/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0511 - val_loss: 0.0542
Epoch 6/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0488 - val_loss: 0.0545
Epoch 7/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0477 - val_loss: 0.0529
Epoch 8/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0462 - val_loss: 0.0522
Epoch 9/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.0439 - val_loss: 0.0521
Epoch 10/10
1136/1136 [==============================] - 6s 6ms/step - loss: 0.043

In [34]:
model4.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0423 - val_loss: 0.0545
Epoch 2/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0411 - val_loss: 0.0533
Epoch 3/10
1136/1136 [==============================] - 8s 7ms/step - loss: 0.0402 - val_loss: 0.0522
Epoch 4/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0387 - val_loss: 0.0529
Epoch 5/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0378 - val_loss: 0.0537
Epoch 6/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0371 - val_loss: 0.0537
Epoch 7/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0358 - val_loss: 0.0541
Epoch 8/10
1136/1136 [==============================] - 7s 7ms/step - loss: 0.0350 - val_loss: 0.0551
Epoch 9/10
1136/1136 [==============================] - 7s 6ms/step - loss: 0.0343 - val_loss: 0.0548
Epoch 10/10
1136/1136 [==============================] - 8s 7ms/step - loss: 0.033

So this seems stuck at 53e-2

In [35]:
model3.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 43s 38ms/step - loss: 0.0318 - val_loss: 0.0423
Epoch 2/10
1136/1136 [==============================] - 51s 45ms/step - loss: 0.0307 - val_loss: 0.0400
Epoch 3/10
1136/1136 [==============================] - 45s 40ms/step - loss: 0.0295 - val_loss: 0.0402
Epoch 4/10
1136/1136 [==============================] - 50s 44ms/step - loss: 0.0285 - val_loss: 0.0402
Epoch 5/10
1136/1136 [==============================] - 56s 49ms/step - loss: 0.0272 - val_loss: 0.0423
Epoch 6/10
1136/1136 [==============================] - 51s 45ms/step - loss: 0.0265 - val_loss: 0.0411
Epoch 7/10
1136/1136 [==============================] - 50s 44ms/step - loss: 0.0256 - val_loss: 0.0403
Epoch 8/10
1136/1136 [==============================] - 47s 42ms/step - loss: 0.0248 - val_loss: 0.0410
Epoch 9/10
1136/1136 [==============================] - 56s 50ms/step - loss: 0.0241 - val_loss: 0.0402
Epoch 10/10
1136/1136 [==============================] - 48s 43m

And this is still stuck at 41e-2

model5 has 4 conv layers

In [36]:
model5 = keras.Sequential()
model5.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [37]:
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 15, 11, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 9, 64)         36928     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 11, 7, 64)         36928     
Total params: 92,960
Trainable params: 92,960
Non-trainable params: 0
_________________________________________________________________


In [38]:
model5.add(layers.Flatten())
model5.add(layers.Dense(256, activation='relu'))
model5.add(layers.Dense(1, activation='relu'))

In [40]:
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 15, 11, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 9, 64)         36928     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 11, 7, 64)         36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 4928)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               1261824   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

A little smaller than model3, but much larger than model 4

In [41]:
model5.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [42]:
model5.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 40s 35ms/step - loss: 0.1210 - val_loss: 0.0666
Epoch 2/10
1136/1136 [==============================] - 41s 36ms/step - loss: 0.0532 - val_loss: 0.0498
Epoch 3/10
1136/1136 [==============================] - 55s 48ms/step - loss: 0.0461 - val_loss: 0.0493
Epoch 4/10
1136/1136 [==============================] - 54s 47ms/step - loss: 0.0434 - val_loss: 0.0439
Epoch 5/10
1136/1136 [==============================] - 58s 51ms/step - loss: 0.0407 - val_loss: 0.0430
Epoch 6/10
1136/1136 [==============================] - 53s 46ms/step - loss: 0.0386 - val_loss: 0.0407
Epoch 7/10
1136/1136 [==============================] - 58s 51ms/step - loss: 0.0380 - val_loss: 0.0395
Epoch 8/10
1136/1136 [==============================] - 63s 56ms/step - loss: 0.0366 - val_loss: 0.0397
Epoch 9/10
1136/1136 [==============================] - 51s 45ms/step - loss: 0.0344 - val_loss: 0.0390
Epoch 10/10
1136/1136 [==============================] - 60s 53m

Hmm. This got the CV error down to 39e-2, but then it jumped back up to 43e-2

model6 has 3 conv layers, but the final one is a little larger so that it has the same number of parameters as model2

In [43]:
model6 = keras.Sequential()
model6.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(layers.Conv2D(90, (3, 3), activation='relu'))

In [44]:
model6.add(layers.Flatten())
model6.add(layers.Dense(256, activation='relu'))
model6.add(layers.Dense(1, activation='relu'))

In [45]:
model6.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 15, 11, 64)        18496     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 13, 9, 90)         51930     
_________________________________________________________________
flatten_3 (Flatten)          (None, 10530)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               2695936   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 2,767,227
Trainable params: 2,767,227
Non-trainable params: 0
____________________________________________

In [46]:
model6.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [47]:
model6.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 54s 47ms/step - loss: 0.1011 - val_loss: 0.0586
Epoch 2/10
1136/1136 [==============================] - 64s 56ms/step - loss: 0.0509 - val_loss: 0.0512
Epoch 3/10
1136/1136 [==============================] - 55s 48ms/step - loss: 0.0455 - val_loss: 0.0448
Epoch 4/10
1136/1136 [==============================] - 50s 44ms/step - loss: 0.0425 - val_loss: 0.0442
Epoch 5/10
1136/1136 [==============================] - 57s 50ms/step - loss: 0.0405 - val_loss: 0.0444
Epoch 6/10
1136/1136 [==============================] - 61s 54ms/step - loss: 0.0384 - val_loss: 0.0424
Epoch 7/10
1136/1136 [==============================] - 50s 44ms/step - loss: 0.0361 - val_loss: 0.0443
Epoch 8/10
1136/1136 [==============================] - 61s 54ms/step - loss: 0.0345 - val_loss: 0.0409
Epoch 9/10
1136/1136 [==============================] - 52s 46ms/step - loss: 0.0329 - val_loss: 0.0415
Epoch 10/10
1136/1136 [==============================] - 51s 45m

In [48]:
model6.fit(train_inputs, train_outputs, epochs=3, validation_split = 0.2)

Epoch 1/3
1136/1136 [==============================] - 38s 34ms/step - loss: 0.0312 - val_loss: 0.0402
Epoch 2/3
1136/1136 [==============================] - 46s 41ms/step - loss: 0.0299 - val_loss: 0.0406
Epoch 3/3
1136/1136 [==============================] - 66s 58ms/step - loss: 0.0286 - val_loss: 0.0415


In [49]:
model7 = keras.Sequential()
model7.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model7.add(layers.Conv2D(64, (3, 3), activation='relu'))
model7.add(layers.Conv2D(90, (3, 3), activation='relu'))

In [50]:
model7.add(layers.Flatten())
model7.add(layers.Dense(256, activation='relu'))
model7.add(layers.Dropout(0.5))
model7.add(layers.Dense(1, activation='relu'))

In [51]:
model7.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 15, 11, 64)        18496     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 13, 9, 90)         51930     
_________________________________________________________________
flatten_4 (Flatten)          (None, 10530)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               2695936   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [52]:
model7.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [53]:
model7.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 40s 35ms/step - loss: 0.1167 - val_loss: 0.0547
Epoch 2/10
1136/1136 [==============================] - 41s 36ms/step - loss: 0.0652 - val_loss: 0.0495
Epoch 3/10
1136/1136 [==============================] - 46s 41ms/step - loss: 0.0563 - val_loss: 0.0455
Epoch 4/10
1136/1136 [==============================] - 55s 49ms/step - loss: 0.0523 - val_loss: 0.0449
Epoch 5/10
1136/1136 [==============================] - 48s 42ms/step - loss: 0.0493 - val_loss: 0.0502
Epoch 6/10
1136/1136 [==============================] - 53s 47ms/step - loss: 0.0482 - val_loss: 0.0416
Epoch 7/10
1136/1136 [==============================] - 51s 45ms/step - loss: 0.0466 - val_loss: 0.0437
Epoch 8/10
1136/1136 [==============================] - 63s 55ms/step - loss: 0.0454 - val_loss: 0.0434
Epoch 9/10
1136/1136 [==============================] - 52s 46ms/step - loss: 0.0442 - val_loss: 0.0423
Epoch 10/10
1136/1136 [==============================] - 52s 46m

In [57]:
model8=keras.Sequential()
model8.add(layers.Dense(256, activation='relu',input_shape=(19, 15, 2)))
model8.add(layers.Dropout(0.5))
model8.add(layers.Dense(1, activation='relu'))

In [58]:
model8.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 19, 15, 256)       768       
_________________________________________________________________
dropout_2 (Dropout)          (None, 19, 15, 256)       0         
_________________________________________________________________
dense_13 (Dense)             (None, 19, 15, 1)         257       
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [52]:
model7.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [53]:
model7.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 40s 35ms/step - loss: 0.1167 - val_loss: 0.0547
Epoch 2/10
1136/1136 [==============================] - 41s 36ms/step - loss: 0.0652 - val_loss: 0.0495
Epoch 3/10
1136/1136 [==============================] - 46s 41ms/step - loss: 0.0563 - val_loss: 0.0455
Epoch 4/10
1136/1136 [==============================] - 55s 49ms/step - loss: 0.0523 - val_loss: 0.0449
Epoch 5/10
1136/1136 [==============================] - 48s 42ms/step - loss: 0.0493 - val_loss: 0.0502
Epoch 6/10
1136/1136 [==============================] - 53s 47ms/step - loss: 0.0482 - val_loss: 0.0416
Epoch 7/10
1136/1136 [==============================] - 51s 45ms/step - loss: 0.0466 - val_loss: 0.0437
Epoch 8/10
1136/1136 [==============================] - 63s 55ms/step - loss: 0.0454 - val_loss: 0.0434
Epoch 9/10
1136/1136 [==============================] - 52s 46ms/step - loss: 0.0442 - val_loss: 0.0423
Epoch 10/10
1136/1136 [==============================] - 52s 46m